# Big Idea #1
When creating computing innovations, developers can follow a structured, step-by-step design process or take a more flexible, experimental approach. Regardless of the method, they will go through key phases such as researching, designing, building prototypes, and testing their solutions. Collaboration is crucial throughout development, as different perspectives contribute to refining and improving innovations for better functionality and usability.


- Debugging was a critical part of building Wellness Waypoints. To ensure the feature worked correctly, I performed backend debugging, frontend debugging, and end-to-end tracing.

## Backend Debugging
Before integrating with the frontend, I tested my API endpoints using Postman. This allowed me to check if my GET, POST, PUT, and DELETE requests were working correctly.

- Example: API GET Request
Here, I sent a GET request to fetch waypoints from the backend.

- The response returns a list of waypoints with information such as address, injury type, and rating.


<img src="../images/get.png" alt="Get Postman" width="500">


## Frontend Debugging
- Once the backend was functional, I tested the API through the frontend. Initially, I encountered authentication errors when making requests.

### Issue: Unauthorized API Access (401 Error)
- The server responded with 401 (Unauthorized) errors because authentication tokens were missing.
- The solution was to add authentication headers to every API request.
Error Log from Console:

<img src="../images/login.png" alt="Login" width="500">

# Frontend to Backend Integration
- To ensure smooth data flow between the frontend and backend, I tested different API calls.

### Example: User Check-in Flow
- User enters an injury type and selects a medical facility.
- The frontend sends a POST request to check-in.
- The check-in appears in the database.
- Data is displayed on the frontend.

<img src="../images/data.png" alt="Data" width="500">

<img src="../images/website.png" alt="Website" width="500">

To validate the system, I have preloaded data in the database already 
```py
def initWaypoints():
    with app.app_context():
        db.create_all()
        test_data = [
            Waypoint(user_id=1, injury='Fracture', location='Hospital', rating=2, address="Naval Medical Center, San Diego"),
            Waypoint(user_id=2, injury='Sprain', location='Clinic', rating=4, address="Scripps Mercy Hospital, San Diego"),
        ]
        
        for entry in test_data:
            try:
                entry.create()
                print(f"Record created: {repr(entry)}")
            except IntegrityError:
                db.session.remove()
                print(f"Record exists or error: {entry.address}")
```

<img src="../images/table.png" alt="Checked In Tables" width="500">


# Big Idea 2 - Managing and Storing Data
To ensure efficient data storage, I used SQLite and enforced data security measures.

### Database Management with SQLite
- Each check-in is stored in the database with:

- ID
- Injury type
- Location
- Rating
- User ID
- Whenever a POST, PUT, or DELETE request is made, the database is updated. 

<img src="../images/database.png" alt="Database" width="700">


# Security & Authentication
### User Role Permissions
- Normal users can only update or delete their own check-ins.
- Admins have the ability to delete any check-in if necessary.

```py
def delete(self):
    current_user = g.current_user
    data = request.get_json()

    if current_user.role == 'Admin':
        waypoint = Waypoint.query.get(data['id'])
        if not waypoint:
            return {'message': 'Waypoint not found'}, 404
        try:
            waypoint.delete()
            return {'message': 'Waypoint deleted successfully'}, 200
        except Exception as e:
            return {'message': f'Error deleting waypoint: {e}'}, 500
    else:
        return {'message': 'Only Admins can delete waypoints'}, 403
```

# Big Idea 4 - Deployment & Security

## Wellness Waypoints was deployed using AWS to make it accessible online.

### Deployment Strategies
- Our team deployed the backend on AWS.
- The frontend runs on GitHub Pages.
- API requests go through a secure domain.
### DNS Configuration
- The backend is mapped to an IP address.
- Users enter the domain name → DNS resolves it to the server IP.


<img src="../images/cockpit.png" alt="Cockpit" width="700">

## HTTP and RESTful APIs
### Wellness Waypoints follows REST API principles:

- GET → Retrieve waypoints
- POST → Check-in at a facility
- PUT → Update rating
- DELETE → Remove check-in (Admin only)
- Example: POST Request in Backend

```py
@token_required()
@cross_origin(supports_credentials=True)
def post(self):
    current_user = g.current_user
    data = request.get_json()

    if not data or 'injury' not in data or 'location' not in data:
        return {'message': 'Injury and location are required'}, 400

    waypoint = Waypoint(
        user_id=current_user.id,
        injury=data.get('injury'),
        location=data.get('location'),
        rating=data.get('rating')
    )

    try:
        waypoint.create()
        return jsonify(waypoint.read())
    except Exception as e:
        return {'message': f'Error saving waypoint: {e}'}, 500
```

## Security Measures
### 1. CORS Configuration
- Only requests from the frontend domain are allowed. My teamate has set this up so that the only domain that can send API requests to the backend is https://kiruthic-selvakumar.github.io


```
location / {
    proxy_pass http://localhost:8101;
    if ($request_method = OPTIONS ) {
        add_header "Access-Control-Allow-Origin" "https://frontend-domain.com" always;
        add_header "Access-Control-Allow-Methods" "GET, POST, PUT, DELETE, OPTIONS" always;
        return 204;
    }
}
```

### 2. HTTPS Enforcement
- Certbot was used to enable SSL encryption.

## Monitoring & Logging
- To track API errors, error handling was implemented.

```py
if not data or 'injury' not in data:
    return {'message': 'Injury type is required'}, 400
```

<img src="../images/carecenter.png" alt="Care Center" width="700">


## Challenges & Solutions
### 1. Real-time Updates
- Issue: The UI was not updating instantly after check-ins.
- Solution: Used asynchronous API calls to refresh data.

### 2. Authentication Issues
- Issue: Users got 401 errors when making API calls.
- Solution: Tokens are now required for all API requests.

### 3. Performance Optimization
- Issue: Large dataset made requests slow.
- Solution: Pagination and lazy loading were added

# Conclusion
- Wellness Waypoints provides a seamless way to find and review medical centers. By following CPT principles, the project now has a robust API, strong security, and a smooth user experience.